In [1]:
import re
import csv
import json
import pickle
import pandas as pd
import xml.etree.ElementTree as ET
from collections import defaultdict
from xml.etree.ElementTree import iterparse

Find all aws-related question ids

In [ ]:
def extract_question_id(file):
    # get an iterable
    context = ET.iterparse(file, events=("start", "end"))

    is_first = True

    question_ids=[]

    for event, elem in context:
        # get the root element
        if is_first:
            root = elem
            is_first = False
        if event == "end" and elem.tag == "row":
            cur_row=elem.attrib
            if cur_row['PostTypeId']=='1' and 'Tags' in cur_row:
                if 'aws' in cur_row['Tags'] or 'amazon' in cur_row['Tags']:
                    question_ids.append(cur_row['Id'])
            root.clear()
    return question_ids


In [ ]:
def write_to_pickle(outfile,question_ids):
    with open(outfile, "wb") as fp:   #Pickling
        pickle.dump(question_ids, fp)

In [ ]:
def read_from_pickle(file):
    with open(file, "rb") as fp:   #Pickling
        question_ids = pickle.load(fp)
    return question_ids

In [ ]:
question_ids=extract_question_id('Posts.xml')
write_to_pickle("question_ids.txt",question_ids)

#### 10/15/2020

1.Use amazon as keyword for searching AWS related posts. <br>
2.Write question information to csv file.

In [20]:
def write_question_to_file(infile,outfile)
    # get an iterable
    context = ET.iterparse(infile, events=("start", "end"))
    is_first = True
    question_list=[]
    for event, elem in context:
        # get the root element
        if is_first:
            root = elem
            is_first = False
        if event == "end" and elem.tag == "row":
            cur_row=elem.attrib
            if cur_row['PostTypeId']=='1' and 'Tags' in cur_row:
                if 'amazon' in cur_row['Tags']:
                    question_list.append(cur_row)
            root.clear()
            
    df=pd.DataFrame(question_list)
    df.to_csv(outfile, sep='\t')

In [21]:
write_question_to_file('Posts.xml','stackoverflow_amazon_q.csv')

#### 10/16/2020

1.Write answer information to csv file. (line by line)

In [34]:
def write_answer_line_by_line(infile,outfile)
    # get an iterable
    context = ET.iterparse(infile, events=("start", "end"))

    is_first = True
    example={'Id': '22467', 'PostTypeId': '2', 'ParentId': '22465', 'CreationDate': '2008-08-22T14:20:02.550', 'Score': '0', 'Body': '<p>Seems like dealing with licensing issues would be nightmarish for the host.</p>\n', 'OwnerUserId': '1902010', 'OwnerDisplayName': 'ceejayoz', 'LastActivityDate': '2008-08-22T14:20:02.550', 'CommentCount': '0', 'ContentLicense': 'CC BY-SA 2.5'}

    question_ids=[]
    #answer_list=[]


    with open(outfile, 'a') as f:
        writer = csv.writer(f)
        #writer.writerow(example.keys())
        for event, elem in context:
            # get the root element
            if is_first:
                root = elem
                is_first = False
            if event == "end" and elem.tag == "row":
                cur_row=elem.attrib
                if cur_row['PostTypeId']=='1' and 'Tags' in cur_row:
                    if 'amazon' in cur_row['Tags']:
                        question_ids.append(cur_row['Id'])
                if cur_row['Id']>'51446634' and cur_row['PostTypeId']=='2' and cur_row['ParentId'] in question_ids:
                    writer.writerow(cur_row.values())
                    #answer_list.append(cur_row)
                root.clear()

In [ ]:
write_answer_line_by_line('Posts.xml','stackoverflow_amazon_a.csv')

#### 10/20/2020
1.write outdated answer information to csv (once).

In [4]:
def get_outdated_answer_id_from_csv(infile)

    answer_ids=[]

    with open(filename, "r") as f:  # on Python 3.x use: open(filename, "r", newline="")
        reader = csv.reader(f)  # create a CSV reader
        header = next(reader)  # grab the first line and keep it as a header reference
        print("CSV header: {}".format(header))
        for row in reader:  # iterate over the available rows
            answer_ids.append(row[0]) 
    return answer_ids

CSV header: ['Id', '', '', '', '', '', '']


In [6]:
def write_outdated_answer_to_file(infile,outfile,answer_ids):

    # get an iterable
    context = ET.iterparse(infile, events=("start", "end"))

    is_first = True


    answer_list=[]



    for event, elem in context:
        # get the root element
        if is_first:
            root = elem
            is_first = False
        if event == "end" and elem.tag == "row":
            cur_row=elem.attrib
            if cur_row['Id'] in answer_ids:
                answer_list.append(cur_row)
            root.clear()
            
    df=pd.DataFrame(answer_list)
    df.to_csv(outfile, sep='\t')

In [ ]:
answer_ids=get_outdated_answer_id_from_csv("outdated_a.csv")
write_outdated_answer_to_file('Posts.xml','stackoverflow_amazon_a_outdated.csv',answer_ids)